In [1]:
import numpy as np
import scipy.signal

In [14]:
def stress_lag_lim_vec_fast(stress_chunk,tstep,samp_freq,lag_len,vol,T):
    
    #conversion from LAMMPS real units to SI units
    
    ang2m = 1*10**-10
    fs2s = 1*10**-15
    kB = 1.3806504e-23 #J/K
    atm2ps = 101325.0
    conv = atm2ps*atm2ps*fs2s*(ang2m**3)
    
    s = []
    inte = []
    
    #sampling of stress components every lag_len timeteps
    
    shift = lag_len/samp_freq
    a = np.arange(0,stress_chunk.shape[0],shift)
    p_xy = np.array([stress_chunk[int(i)][0] for i in a])
    p_xz = np.array([stress_chunk[int(i)][1] for i in a])
    p_yz = np.array([stress_chunk[int(i)][2] for i in a])
    p_diff_1 = np.array([stress_chunk[int(i)][3] for i in a])
    p_diff_2 = np.array([stress_chunk[int(i)][4] for i in a])
    
    #main code to calculate autocorrelation of stress followed by ensemble averaging 
    
    acf_temp_xy = scipy.signal.correlate(p_xy,p_xy)
    acf_temp_xz = scipy.signal.correlate(p_xz,p_xz)
    acf_temp_yz = scipy.signal.correlate(p_yz,p_yz)
    acf_temp_diff_1 = scipy.signal.correlate(p_diff_1,p_diff_1)
    acf_temp_diff_2 = scipy.signal.correlate(p_diff_2,p_diff_2)
    
    acf_xy = np.flip(acf_temp_xy[0:len(p_xy)])
    acf_xz = np.flip(acf_temp_xz[0:len(p_xz)])
    acf_yz = np.flip(acf_temp_yz[0:len(p_yz)])
    acf_diff_1 = np.flip(acf_temp_diff_1[0:len(p_diff_1)])
    acf_diff_2 = np.flip(acf_temp_diff_2[0:len(p_diff_2)])
    acf_tot = acf_xy + acf_xz + acf_yz + acf_diff_1 + acf_diff_2
    
    norm = np.flip(np.arange(1,len(p_xy)+1,1))
    acf_norm = acf_tot/norm
    
    #integration of autocorrelation function to calculate viscosity in SI units
    
    for i in range(len(acf_tot)):
        temp = np.trapz(acf_norm[0:i+1],dx=(lag_len*tstep))
        temp_s = (conv*temp*vol)/(5*kB*T)
        inte.append(temp)
        s.append(temp_s)
       
    return acf_norm,inte,s

In [15]:
filename = '/Users/rajorshi/Desktop/CMD/Conductivity/Reaxff/673K_5GPa_new/Viscousity/input.txt'        #path of input stress data file
filename_prog = '/Users/rajorshi/Desktop/CMD/Conductivity/Reaxff/673K_5GPa_new/Viscousity/output.dat'  #path of output data file
f_prog = open(filename_prog,'w')

#load data into numpy arrays

stress = np.loadtxt(filename,skiprows=2,usecols=(1,2,3,4,5,6))
t = np.loadtxt(filename,skiprows=2,usecols=(0))
stress[:,3],stress[:,4] =  (stress[:,3]-stress[:,4])/2,(stress[:,4]-stress[:,5])/2

    
samp_freq = 2      #sampling frequency in timesteps
lag_len = 2        #difference between successive lag times (in timesteps) where autocorrelation is calculated. Has to be multiple of sampling frequency
time_chunk = 2     #input data is broken down into chunks of size time_chunk (in picosecond). Each chunk can then be used for analysis
tstep = 0.5        #timestep of MD simulations

#simulation parameters

box_max = 28.688099
box_min = 3.311927
vol = (box_max-box_min)**3
T = 673

#extract stress data from input file in chunks of size time_chunk and store them in dictionary

steps = time_chunk*1000*(1/tstep)
steps_arr = steps/samp_freq

n_chunk = 2      #number of time chunks of size time_chunk for stress calculation. Important for assessing convergence wrt simulation length
stress_chunk = {}
start = int(len(stress)-steps_arr)
end = int(len(stress))

#write out important simulation parameters in output file

f_prog.write('Box size: max: '+str(box_max)+' min: '+str(box_min)+' in Ang'+'\n')
f_prog.write('Timestep: '+str(tstep)+'\n')
f_prog.write('T: '+str(T)+'\n')

#number of timesteps in input.dat file must be atleast time_chunk*t_step*n_chunk
while start>=0:
    temp = stress[start-1:end][:]
    stress_chunk.update({t[start-1]:temp})
    end = int(start)
    start = int(end - steps_arr)

count = 0

#iterate over dictionary entries and calculate viscosity for n_chunk time chunks

for key in stress_chunk.keys():
    lag = 0
    count = count + 1
    
    if count <= n_chunk:
        print('Calculating for chunck {0}'.format(count))
        
        acf_norm,inte,s = stress_lag_lim_vec_fast(stress_chunk[key],tstep,samp_freq,lag_len,vol,T)
        #print(v)
        f_prog.write('Chunk '+str(count)+' of size '+str(time_chunk)+'ps'+'\n')
        f_prog.write('Start time: '+str((int(key)*tstep)/1000)+'ps'+'\n')
        f_prog.write('Lag'+'    '+'Lag (in ps)'+'     '+'Auto-correlation'+'     '+'Integral(acf)'+'        '+'Viscosity(in Pa s)'+'\n')
        for i in range(len(acf_norm)):
            f_prog.write(str(lag)+'      '+str((lag*tstep)/1000)+'          '+str(acf_norm[i])+'    '+str(inte[i])+'    '+str(s[i])+'\n') 
            lag = lag + lag_len
        
f_prog.close()

Calculating for chunck 1
Calculating for chunck 2
